In [1]:
# Import relevant modules

%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import copy

import pandas as pd
import numpy as np
import healpy as hp
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.optimize import minimize
from astropy.io import fits
from tqdm import *
import matplotlib.pyplot as plt
import iminuit
from iminuit import Minuit, describe, Struct

# NPTFit modules
from NPTFit import nptfit # module for performing scan
from NPTFit import create_mask as cm # module for creating the mask
from NPTFit import psf_correction as pc # module for determining the PSF correction

ImportError: libimf.so: cannot open shared object file: No such file or directory

In [ ]:
FermiData = np.load('fermi_data/fermidata_counts.npy').astype(np.int32)
fermi_exposure = np.load('fermi_data/fermidata_exposure.npy')
dif = np.load('fermi_data/template_dif.npy')
iso = np.load('fermi_data/template_iso.npy')
psc = np.load('fermi_data/template_psc.npy')
subhalos = np.load('EinastoTemplate.npy')
subhalos = subhalos
hp.mollview(subhalos)

In [ ]:
n = nptfit.NPTF(tag='norm')
n.load_data(FermiData, fermi_exposure)

pscmask=np.array(np.load('fermi_data/fermidata_pscmask.npy'), dtype=bool)
mask = cm.make_mask_total(band_mask = True, band_mask_range = 5, mask_ring = True, inner = 20, outer = 180, custom_mask = pscmask)
n.load_mask(mask)

n.add_template(dif, 'dif')
n.add_template(iso, 'iso')
n.add_template(psc, 'psc')

n.add_poiss_model('dif', '$A_\mathrm{dif}$', [0,10], False)
n.add_poiss_model('iso', '$A_\mathrm{iso}$', [0,20], False)
n.add_poiss_model('psc', '$A_\mathrm{psc}$', [0,10], False)

xsec0 = 1e-24
A0 = 10**(-13.9442)
n10 = 0.0238081
n20 = 10
Fb0 = 10**(-9.71429)

xsec_arr = np.logspace(-24, -25, 1)

LL_xsec_ary = np.zeros(len(xsec_arr))

for ix, xsec in enumerate(xsec_arr):
    A = A0 * xsec/xsec0
    Fb = Fb0 * xsec0/xsec

    new_n = copy.copy(n)
    #new_n.add_template(subhalos, 'subhalos', units='PS')
    #new_n.add_poiss_model('subhalos', '$A_\mathrm{sub}$', fixed=True)
    #new_n.add_non_poiss_model('subhalos', 
    #                       ['$A^\mathrm{ps}_\mathrm{iso}$','$n_1$','$n_2$','$F_b$'],
    #                       fixed_params = [[0, A], [1, n10], [2, n20], [3,Fb]])
    new_n.configure_for_scan()
    scpy_min = minimize(lambda x: -new_n.ll(x) ,x0=[1.,1.,1.], bounds=[[0,30.], [0,30.], [0,30.]], options={'disp': False, 'ftol':1e-12}, method='SLSQP')
    max_LL = -scpy_min['fun']
    best_fit_params = scpy_min['x']
    print("Template best-fit params are", best_fit_params)
    print("Max L-L is", max_LL)
    LL_xsec_ary[ix] = max_LL

In [ ]:
fake_data = 13.9583217*dif + 1.06289421*iso + 0.90448092*psc
fake_data += np.load("subhalo_flux_map.npy")*fermi_exposure
fake_data = np.random.poisson(fake_data)
fake_data = fake_data.astype(np.int32)

In [ ]:
n = nptfit.NPTF(tag='norm')
n.load_data(fake_data, fermi_exposure)
pscmask=np.array(np.load('fermi_data/fermidata_pscmask.npy'), dtype=bool)
mask = cm.make_mask_total(band_mask = True, band_mask_range = 5, mask_ring = True, inner = 20, outer = 180, custom_mask = pscmask)
n.load_mask(mask)

n.add_template(dif, 'dif')
n.add_template(iso, 'iso')
n.add_template(psc, 'psc')

n.add_poiss_model('dif', '$A_\mathrm{dif}$', [0,10], False)
n.add_poiss_model('iso', '$A_\mathrm{iso}$', [0,20], False)
n.add_poiss_model('psc', '$A_\mathrm{psc}$', [0,10], False)

xsec0 = 1e-24
A0 = 10**(-13.9482)
n10 = -0.0309105
n20 = -10
Fb0 = 10**(-9.71429)

xsec_arr = np.logspace(-26, -20, 15)
LL_xsec_ary = np.zeros(len(xsec_arr))

for ix, xsec in tqdm_notebook(enumerate(xsec_arr)):
    A = A0 * xsec/xsec0
    Fb = Fb0 * xsec/xsec0

    new_n = copy.copy(n)
    new_n.add_template(subhalos, 'subhalos', units='PS')
    new_n.add_poiss_model('subhalos', '$A_\mathrm{sub}$', fixed=True)
    new_n.add_non_poiss_model('subhalos', 
                           ['$A^\mathrm{ps}_\mathrm{iso}$','$n_1$','$n_2$','$F_b$'],
                           fixed_params = [[0, A], [1, n10], [2, n20], [3,Fb]], units='flux')
    new_n.configure_for_scan()
    minuit_min = iminuit.Minuit(lambda d, i, p: -new_n.ll([d, i, p]) , d=0.1, i=0.1, p=0.1, limit_d=(0.,30.), limit_i=(0.,30.), limit_p=(0.,30.), error_d=1.0, error_i=1.0, error_p=1.0, print_level=0);
    minuit_min.migrad()
    max_LL = -minuit_min.fval
    best_fit_params = [minuit_min.values]
    print("Best Fit Params:", best_fit_params)
    LL_xsec_ary[ix] = max_LL

In [ ]:
hp.mollview(np.log10(1e-30+fakedata), title="Mock Data")

In [ ]:
plt.plot(xsec_arr, LL_xsec_ary)
print(np.argmax(LL_xsec_ary))
#plt.ylim(-1e5, 1e5)
print(LL_xsec_ary)
plt.xscale("log")
#plt.yscale("log")